In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\cjcre\\Downloads\\application_default_credentials.json"

In [ ]:
!pip install google-cloud-documentai google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas vertexai

In [ ]:
import email
from email.utils import parsedate_to_datetime
from datetime import datetime
from google.cloud import documentai
from google.api_core.client_options import ClientOptions
from IPython.display import display, HTML
import pandas as pd
from imaplib import IMAP4_SSL
import vertexai
from vertexai.generative_models import (
    GenerativeModel,
    SafetySetting,)
import json
project_id = "invoiceprocessing-450716"
location = "us"
processor_id = "5486fff89ef396e2"
mime_type = "application/pdf"
field_mask = "text,entities,pages.pageNumber"
processor_version_id = "57f4f1dda43d5c50"
def get_gmail_service(username, password):
    try:
        mail = IMAP4_SSL("imap.gmail.com")
        mail.login(username, password)
        mail.select("inbox")
        return mail
    except IMAP4_SSL.error as e:
        print(f"IMAP Error: {e}")
        return None
    except Exception as e:
        print(f"Error getting Gmail service: {e}")
        return None
def process_document_from_memory(
    image_content,
    mime_type,
    project_id,
    location,
    processor_id,
    field_mask=None,
    processor_version_id=None,):
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")
    client = documentai.DocumentProcessorServiceClient(client_options=opts)
    if processor_version_id:
        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id)
    else:
        name = client.processor_path(project_id, location, processor_id)
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)
    process_options = documentai.ProcessOptions(
        individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
            pages=[1]))
    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document,
        field_mask=field_mask,
        process_options=process_options,)
    result = client.process_document(request=request)
    document = result.document
    return document
def process_email(gmail_service, msg, num, label_name):
    date_str = msg["Date"]
    email_datetime = parsedate_to_datetime(date_str) if date_str else datetime.now()
    extracted_data = []
    for part in msg.walk():
        if part.get_content_maintype() == "multipart":
            continue
        if part.get("Content-Disposition") is None:
            continue
        filename = part.get_filename()
        if filename and filename.lower().endswith(".pdf"):
            payload = part.get_payload(decode=True)
            pdf_title_text = "No Title Found"
            try:
                document = process_document_from_memory(
                    image_content=payload,
                    mime_type="application/pdf",
                    project_id=project_id,
                    location=location,
                    processor_id=processor_id,
                    field_mask="text",
                    processor_version_id=processor_version_id,
                )
                if document.text:
                    pdf_title_text = " ".join(document.text.split()[:10]) + "..."
                text = document.text
                entities = document.entities
                extracted_entities = {}
                for entity in entities:
                    extracted_entities[entity.type_] = entity.mention_text
                document_data = {
                    "text": text,
                    "entities": [
                        {"type_": e.type_, "mention_text": e.mention_text} for e in entities],
                }
                extracted_data.append(
                    {
                        "filename": filename,
                        "email_datetime": email_datetime,
                        "text": text,
                        "entities": extracted_entities,
                        "document_json": json.dumps(document_data),
                        "gmail_search_query": f"in:inbox \"{pdf_title_text}\"",
                    }
                )
                print(f"Extracted from {filename}: {text[:100]}...")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    try:
        status, data = gmail_service.store(num, "+X-GM-LABELS", label_name)
        if status == "OK":
            print(f"Applied label '{label_name}' to email {num}")
        else:
            print(f"Failed to apply label '{label_name}' to email {num}")
    except Exception as e:
        print(f"Error applying label: {e}")
    print(f"Processed email from {email_datetime}")
    return extracted_data
def process_gmail_pdfs(username, password):
    gmail_service = get_gmail_service(username, password)
    if gmail_service is None:
        return
    all_extracted_data = []
    try:
        _, data = gmail_service.search(None, "UNSEEN", "ALL")
        mail_ids = data[0]
        id_list = mail_ids.split()
        now = datetime.now()
        label_name = now.strftime("%B_%Y_Invoices")
        status, data = gmail_service.list()
        if status == "OK":
            label_exists = False
            for item in data:
                if label_name in str(item):
                    label_exists = True
                    break
            if not label_exists:
                status, data = gmail_service.create('"' + label_name + '"')
                if status == "OK":
                    print(f"Created label '{label_name}'")
                else:
                    print(f"Failed to create label '{label_name}'")
        else:
            print("Failed to list labels.")
        for num in id_list:
            _, data = gmail_service.fetch(num, "(RFC822)")
            for response_part in data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    extracted_data = process_email(
                        gmail_service, msg, num, label_name)
                    all_extracted_data.extend(extracted_data)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        try:
            gmail_service.close()
            gmail_service.logout()
        except:
            pass
    return all_extracted_data
def display_extracted_data(extracted_data):
    if not extracted_data:
        print("No data extracted from emails.")
        return
    data_list = []
    for data in extracted_data:
        data_row = {
            "Filename": data["filename"],
            "Email Date": data["email_datetime"],
            "Text": data["text"][:200] + "...",}
        data_row.update(data["entities"])
        data_row["Document JSON"] = data[
            "document_json"]
        data_list.append(data_row)
    df = pd.DataFrame(data_list)
    display(HTML(df.to_html(index=False)))
def multiturn_generate_content(document_json_string):
    vertexai.init(
        project="invoiceprocessing-450716",
        location="us-central1",
        api_endpoint="aiplatform.googleapis.com",)
    model = GenerativeModel(
        "gemini-1.5-pro-002", system_instruction=[si_text1])
    chat = model.start_chat()
    try:
        response = chat.send_message(
            [msg1_text1.replace("[Value from Doc AI]", document_json_string)],
            generation_config=generation_config,
            safety_settings=safety_settings,)
        return response.text
    except Exception as e:
        print(f"Error calling Vertex AI: {e}")
        return None
msg1_text1 = """General Information Verification:
\"Google Document AI extracted the Invoice Number as '[Value from Doc AI]'. Please verify this against the attached PDF. If it matches, respond 'Match'. If not, provide the correct Invoice Number from the PDF.\" (Example source )
\"Google Document AI extracted the Invoice Date as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Invoice Date.\" (Example source )
\"Google Document AI extracted the Due Date as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Due Date.\" (Example source )
\"Google Document AI extracted the Vendor Name as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Vendor Name.\" (Example source )
\"Google Document AI extracted the Bill To Name as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Bill To Name.\" (Example source )
\"Google Document AI extracted the Ship To Name as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Ship To Name.\" (Example source )
Financial Details Verification:
\"Google Document AI extracted the Subtotal as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Subtotal.\" (Example source )
\"Google Document AI extracted the Sales Tax as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Sales Tax amount.\" (Example source )
\"Google Document AI extracted the Total Amount Due as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Total Amount Due.\" (Example source )
\"Google Document AI extracted the Balance Due as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Balance Due.\" (Example source )
Line Item Verification (Repeat for each relevant line item):
\"For the line item where Google Document AI extracted the Description as '[Value from Doc AI]', verify this description against the PDF. If match, respond 'Match'. If not, provide the correct Description.\" (Example source )
\"For the line item with description '[Verified Description from PDF]', Google Document AI extracted the Quantity as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Quantity.\" (Example source )
\"For the line item with description '[Verified Description from PDF]', Google Document AI extracted the Unit Price as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Unit Price.\" (Example source )
\"For the line item with description '[Verified Description from PDF]', Google Document AI extracted the Total Line Amount as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Total Line Amount.\" (Example source )
Statement Specific Verification:
\"Google Document AI extracted the Statement Date as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Statement Date.\" (Example source )
\"Google Document AI extracted the final Amount Due (from the summary/aging) as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Amount Due.\" (Example source )"""
si_text1 = """You are an AI assistant functioning as a data validation and verification specialist. Your task is to compare data extracted by Google Document AI against the original source document (invoice or statement PDF).
Your Task:
You will be provided with a specific data field (e.g., \"Invoice Number\") and the value extracted for that field by Google Document AI.
You will also be given access to, or the text content of, the original source PDF document (invoice or statement).
Carefully examine the source PDF document to locate the specified data field.
Compare the value present in the PDF document with the value provided by Google Document AI.
Verification:If the value from Google Document AI matches the value in the PDF, confirm the match.
If the value from Google Document AI does not match the value in the PDF, provide the correct value as it appears in the PDF.
If the specified data field cannot be found in the PDF document, state that it is not present.
Focus solely on verifying the provided data points against the visual information in the PDF. Do not infer, calculate, or validate information beyond this comparison.
I need the vertex ai response to have standardized names, so for invoice number always use invoice number throughout all the documents. I want all the headers to be standardized.
I want to see the value found instead of match.
For line item verification, I want the name of the line item to be Description 1, Description 2, and so on with there also be a matching Quantity 1, Unit Price 1, Total Amount 1."""
generation_config = {
    "max_output_tokens": 2048,
    "temperature": 0.2,
    "top_p": 0.95,}
safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF,),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF,),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF,),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF,),]
if __name__ == "__main__":
    username = "lucky.lad.test.df@gmail.com"
    password = "elsu cbfl urqv ahok"
    extracted_data = process_gmail_pdfs(username, password)
    vertex_ai_responses = {}
    cleaned_responses = {}
    if extracted_data:
        for i, data in enumerate(extracted_data):
            document_json_string = data["document_json"]
            vertex_ai_response = multiturn_generate_content(document_json_string)
            if vertex_ai_response:
                response_variable_name = f"vertex_ai_response_{i + 1}"
                vertex_ai_responses[response_variable_name] = vertex_ai_response
                print(f"Vertex AI Response for document {i + 1}:\n")
            else:
                print(f"Vertex AI call failed for document {i + 1}")
    else:
        print("No data extracted from emails.")
    for response_name, response_content in vertex_ai_responses.items():
        cleaned_response = "\n".join(line.replace("Value found: ", "").replace("* ", "").lstrip("*") for line in response_content.splitlines())
        cleaned_response = "\n".join(line.replace("*", "").strip() for line in cleaned_response.splitlines())
        cleaned_responses[response_name] = cleaned_response
        print(f"--- Cleaned {response_name} ---")
        print()
    dataframes = {}
    df_counter = 1
    prefixes_to_remove = [
        "GeneralInformationVerification_",
        "FinancialDetailsVerification_",
        "StatementSpecificVerification_",
        "LineItemVerification_",]
    for i, (response_name, response_content) in enumerate(cleaned_responses.items()):
        data = {}
        current_section = None
        line_item_counter = 1
        gmail_search_query = None
        pdf_filename = None
        if extracted_data and i < len(extracted_data):
            gmail_search_query = extracted_data[i].get("gmail_search_query")
            pdf_filename = extracted_data[i].get("filename")
        for line in response_content.splitlines():
            line = line.strip()
            if line.endswith("Verification:"):
                current_section = line[:-1].strip()
                if current_section not in data:
                    data[current_section] = {}
                line_item_counter = 1
            elif ":" in line:
                key, value = line.split(":", 1)
                key = key.strip()
                value = value.strip()
                if current_section == "Line Item Verification":
                    if key.startswith("Description"):
                        data[current_section][f"Description {line_item_counter}"] = value
                    elif key.startswith("Quantity"):
                        data[current_section][f"Quantity {line_item_counter}"] = value
                    elif key.startswith("Unit Price"):
                        data[current_section][f"Unit Price {line_item_counter}"] = value
                    elif key.startswith("Total Amount"):
                        data[current_section][f"Total Amount {line_item_counter}"] = value
                        line_item_counter += 1
                    else:
                        data[current_section][key] = value
                else:
                    data[current_section][key] = value
        df_data = {}
        for section, values in data.items():
            for key, value in values.items():
                df_data[f"{section.replace(' ', '')}_{key.replace(' ', '')}"] = value
        df_data["PDF Filename"] = pdf_filename
        df = pd.DataFrame([df_data])
        new_columns = {}
        for col in df.columns:
            new_col = col
            for prefix in prefixes_to_remove:
                new_col = new_col.replace(prefix, "")
            new_columns[col] = new_col
        df = df.rename(columns=new_columns)
        df_name = f"df{df_counter}"
        globals()[df_name] = df
        dataframes[response_name] = globals()[df_name]
        df_counter += 1
    final_df = pd.DataFrame()
    for df_name, df in dataframes.items():
        if final_df.empty:
            final_df = df
        else:
            final_df = pd.concat([final_df, df], ignore_index=True, sort=False)
    print("\n--- Final Concatenated DataFrame (final_df) ---")
final_df = final_df.rename(columns={'AmountDue(fromthesummary/aging)': 'AmountDuefromthesummary_aging'})
final_df = final_df.rename(columns={'PDF Filename': 'PDFFilename'})
final_df = final_df.rename(columns={'FinalAmountDue(fromthesummary/aging)': 'FinalamountDuefromthesummary_aging'})
final_df = final_df.astype(str)
print(final_df)
import snowflake.connector
SNOWFLAKE_ACCOUNT = "ifb67743.us-east-1"
SNOWFLAKE_USER = "DF_SVC_US_WELLS_UPDATE"
SNOWFLAKE_PASSWORD = "g]=V41$;2(Aj3s#3"
SNOWFLAKE_DATABASE = "OCCLUSION"
SNOWFLAKE_SCHEMA = "WELLS"
SNOWFLAKE_WAREHOUSE = "COMPUTE_WH"
SNOWFLAKE_TABLE = "LUCKY_LAD_INVOICE_PROCESSOR"
def add_missing_columns(df: pd.DataFrame):
    ctx = None
    cs = None
    try:
        ctx = snowflake.connector.connect(
            account=SNOWFLAKE_ACCOUNT,
            user=SNOWFLAKE_USER,
            password=SNOWFLAKE_PASSWORD,
            database=SNOWFLAKE_DATABASE,
            schema=SNOWFLAKE_SCHEMA,
            warehouse=SNOWFLAKE_WAREHOUSE)
        cs = ctx.cursor()
        df_columns = set(df.columns)
        table_columns_sql = f"""
        SELECT column_name
        FROM {SNOWFLAKE_DATABASE}.INFORMATION_SCHEMA.COLUMNS
        WHERE table_catalog = '{SNOWFLAKE_DATABASE}'
          AND table_schema = '{SNOWFLAKE_SCHEMA}'
          AND table_name = '{SNOWFLAKE_TABLE}'
        """
        cs.execute(table_columns_sql)
        existing_columns = {row[0].upper() for row in cs.fetchall()}
        new_columns_to_add = df_columns - existing_columns
        for col in new_columns_to_add:
            alter_table_sql = f"""
            ALTER TABLE {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.{SNOWFLAKE_TABLE}
            ADD COLUMN {col} VARCHAR;
            """
            try:
                cs.execute(alter_table_sql)
                print(f"Added column '{col}' to table '{SNOWFLAKE_TABLE}'.")
            except snowflake.connector.errors.ProgrammingError as e:
                print(f"Error adding column '{col}': {e}")
            except snowflake.connector.errors.DatabaseError as e:
                print(f"Database Error adding column '{col}': {e}")
            except Exception as e:
                print(f"General error adding column '{col}': {e}")
        ctx.commit()
    except snowflake.connector.errors.Error as e:
        print(f"Snowflake Error in add_missing_columns: {e}")
    finally:
        if cs:
            cs.close()
        if ctx:
            ctx.close()
def add_dataframe_to_snowflake(df: pd.DataFrame):
    ctx = None
    cs = None
    try:
        ctx = snowflake.connector.connect(
            account=SNOWFLAKE_ACCOUNT,
            user=SNOWFLAKE_USER,
            password=SNOWFLAKE_PASSWORD,
            database=SNOWFLAKE_DATABASE,
            schema=SNOWFLAKE_SCHEMA,
            warehouse=SNOWFLAKE_WAREHOUSE)
        cs = ctx.cursor()
        add_missing_columns(df)
        columns = ", ".join(df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.{SNOWFLAKE_TABLE} ({columns}) VALUES ({placeholders})"
        for row in df.itertuples(index=False):
            try:
                cs.execute(sql, row)
            except snowflake.connector.errors.ProgrammingError as e:
                print(f"Error inserting row: {row}")
                print(f"Snowflake Programming Error: {e}")
            except Exception as e:
                print(f"General error inserting row: {row}")
                print(f"Python Error: {e}")
        ctx.commit()
        print(f"Successfully added {len(df)} rows to {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.{SNOWFLAKE_TABLE}")
    except snowflake.connector.errors.Error as e:
        print(f"Snowflake Error in add_dataframe_to_snowflake: {e}")
    finally:
        if ctx:
            cs.close()
            ctx.close()
if __name__ == "__main__":
    add_dataframe_to_snowflake(final_df)
    print("Script finished.")

In [ ]:
msg1_text1 = """General Information Verification:
\"Google Document AI extracted the Invoice Number as '[Value from Doc AI]'. Please verify this against the attached PDF. If it matches, respond 'Match'. If not, provide the correct Invoice Number from the PDF.\" (Example source )
\"Google Document AI extracted the Invoice Date as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Invoice Date.\" (Example source )
\"Google Document AI extracted the Due Date as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Due Date.\" (Example source )
\"Google Document AI extracted the Vendor Name as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Vendor Name.\" (Example source )
\"Google Document AI extracted the Bill To Name as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Bill To Name.\" (Example source )
\"Google Document AI extracted the Ship To Name as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Ship To Name.\" (Example source )
Financial Details Verification:
\"Google Document AI extracted the Subtotal as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Subtotal.\" (Example source )
\"Google Document AI extracted the Sales Tax as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Sales Tax amount.\" (Example source )
\"Google Document AI extracted the Total Amount Due as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Total Amount Due.\" (Example source )
\"Google Document AI extracted the Balance Due as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Balance Due.\" (Example source )
Line Item Verification (Repeat for each relevant line item):
\"For the line item where Google Document AI extracted the Description as '[Value from Doc AI]', verify this description against the PDF. If match, respond 'Match'. If not, provide the correct Description.\" (Example source )
\"For the line item with description '[Verified Description from PDF]', Google Document AI extracted the Quantity as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Quantity.\" (Example source )
\"For the line item with description '[Verified Description from PDF]', Google Document AI extracted the Unit Price as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Unit Price.\" (Example source )
\"For the line item with description '[Verified Description from PDF]', Google Document AI extracted the Total Line Amount as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Total Line Amount.\" (Example source )
Statement Specific Verification:
\"Google Document AI extracted the Statement Date as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Statement Date.\" (Example source )
\"Google Document AI extracted the final Amount Due (from the summary/aging) as '[Value from Doc AI]'. Verify against the PDF. If match, respond 'Match'. If not, provide the correct Amount Due.\" (Example source )"""
si_text1 = """You are an AI assistant functioning as a data validation and verification specialist. Your task is to compare data extracted by Google Document AI against the original source document (invoice or statement PDF).
Your Task:
You will be provided with a specific data field (e.g., \"Invoice Number\") and the value extracted for that field by Google Document AI.
You will also be given access to, or the text content of, the original source PDF document (invoice or statement).
Carefully examine the source PDF document to locate the specified data field.
Compare the value present in the PDF document with the value provided by Google Document AI.
Verification:If the value from Google Document AI matches the value in the PDF, confirm the match.
If the value from Google Document AI does not match the value in the PDF, provide the correct value as it appears in the PDF.
If the specified data field cannot be found in the PDF document, state that it is not present.
Focus solely on verifying the provided data points against the visual information in the PDF. Do not infer, calculate, or validate information beyond this comparison.
I need the vertex ai response to have standardized names, so for invoice number always use invoice number throughout all the documents. I want all the headers to be standardized.
I want to see the value found instead of match.
For line item verification, I want the name of the line item to be Description 1, Description 2, and so on with there also be a matching Quantity 1, Unit Price 1, Total Amount 1."""
